## Задание 9.5. Модуль ML-6 (HW-03)

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_selection import RFE, SelectKBest

from sklearn import metrics #метрики

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [4]:
y = data['price']
X = data.drop(columns='price')

## Предобработка. Заполнение пропусков
Попробуем заполнить пропуски колонок drive, size и weather, используя ML

In [5]:
# для начала закодируем категориальные признаки
categorial_cols = ['cylinders', 'title_status', 'transmission']
X = pd.get_dummies(X, columns=categorial_cols)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    7017 non-null   int64  
 1   condition               7017 non-null   int64  
 2   odometer                7017 non-null   int64  
 3   drive                   6626 non-null   object 
 4   size                    5453 non-null   object 
 5   lat                     7017 non-null   float64
 6   long                    7017 non-null   float64
 7   weather                 6837 non-null   float64
 8   cylinders_3             7017 non-null   uint8  
 9   cylinders_4             7017 non-null   uint8  
 10  cylinders_5             7017 non-null   uint8  
 11  cylinders_6             7017 non-null   uint8  
 12  cylinders_8             7017 non-null   uint8  
 13  cylinders_10            7017 non-null   uint8  
 14  title_status_clean      7017 non-null   

Для начала заполним колонку weather

In [6]:
data = X.copy()
 
weather_data = data[data['weather'].isnull()] # получим таблицу, со всеми пустыми значениями weather, сохранив индексы
size_data = data[data['size'].isnull()] # эти таблицы будут нашими "тестовыми", а индексы будут нужны, что-бы по ним вставить
drive_data = data[data['drive'].isnull()] # предсказанные значения

data.dropna(inplace=True) # получим таблицу, где нет пустых значений вообще. Никаких
 
y_train_weather = data['weather']  # наш целевой признак weather
y_train_size = data['size']
y_train_drive = data['drive']

X_train = data.drop(['size','weather','drive'], axis=1) # тренировочная таблица одна на всех

X_test_weather = weather_data.drop(['size','weather','drive'], axis=1)
X_test_size = size_data.drop(['size','weather','drive'], axis=1)
X_test_drive = drive_data.drop(['size','weather','drive'], axis=1)

In [7]:
# Предскажем Weather с помощью линейной регрессии
model = LinearRegression()

model.fit(X_train, y_train_weather)
y_pred_weather = model.predict(X_test_weather) # получим предсказания пропущенных значений

# Предскажем drive и size с помощью логистической регрессии

model = LogisticRegression(max_iter=1000)

model.fit(X_train, y_train_drive)
y_pred_drive = model.predict(X_test_drive)

model.fit(X_train, y_train_size)
y_pred_size = model.predict(X_test_size)

In [8]:
# Заполняем пропуски
for i, ni in enumerate(weather_data.index):
    #print (i,ni)
    X.loc[ni, 'weather'] = y_pred_weather[i]

for i, ni in enumerate(size_data.index):
    X.loc[ni, 'size'] = y_pred_size[i]

for i, ni in enumerate(drive_data.index):
    X.loc[ni, 'drive'] = y_pred_drive[i]

In [9]:
# Проверяем
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    7017 non-null   int64  
 1   condition               7017 non-null   int64  
 2   odometer                7017 non-null   int64  
 3   drive                   7017 non-null   object 
 4   size                    7017 non-null   object 
 5   lat                     7017 non-null   float64
 6   long                    7017 non-null   float64
 7   weather                 7017 non-null   float64
 8   cylinders_3             7017 non-null   uint8  
 9   cylinders_4             7017 non-null   uint8  
 10  cylinders_5             7017 non-null   uint8  
 11  cylinders_6             7017 non-null   uint8  
 12  cylinders_8             7017 non-null   uint8  
 13  cylinders_10            7017 non-null   uint8  
 14  title_status_clean      7017 non-null   

### Теперь закодируем drive и size

In [10]:
categorial_cols = ['drive', 'size']
X = pd.get_dummies(X, columns=categorial_cols)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    7017 non-null   int64  
 1   condition               7017 non-null   int64  
 2   odometer                7017 non-null   int64  
 3   lat                     7017 non-null   float64
 4   long                    7017 non-null   float64
 5   weather                 7017 non-null   float64
 6   cylinders_3             7017 non-null   uint8  
 7   cylinders_4             7017 non-null   uint8  
 8   cylinders_5             7017 non-null   uint8  
 9   cylinders_6             7017 non-null   uint8  
 10  cylinders_8             7017 non-null   uint8  
 11  cylinders_10            7017 non-null   uint8  
 12  title_status_clean      7017 non-null   uint8  
 13  title_status_lien       7017 non-null   uint8  
 14  title_status_missing    7017 non-null   

## Отбор признаков


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### По методу RFE

In [12]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['size_compact', 'size_full-size', 'size_sub-compact'], dtype=object)

Размер - является "сильным" признаком. Но в отбор попали 3 столбца одного признака. Увеличим размер n_features_to_select, что-бы увидеть 3 основных признака.

In [13]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=8, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['cylinders_3', 'cylinders_6', 'cylinders_10',
       'transmission_manual', 'size_compact', 'size_full-size',
       'size_mid-size', 'size_sub-compact'], dtype=object)

Итак, по факту, по этому методу у нас 3 важных признака это: cylinders, transmission, size

### По методу SelectKBest

Поскольку искомый признак - числовой, значит нам подойдёт метод на основе коэффициента корреляции Пирсона

In [14]:
from sklearn.feature_selection import f_regression

In [15]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'odometer'], dtype=object)

В этом случае наиболее важными оказались признаки year, condition и odometer

## Обучение модели

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X.columns

Index(['year', 'condition', 'odometer', 'lat', 'long', 'weather',
       'cylinders_3', 'cylinders_4', 'cylinders_5', 'cylinders_6',
       'cylinders_8', 'cylinders_10', 'title_status_clean',
       'title_status_lien', 'title_status_missing', 'title_status_rebuilt',
       'title_status_salvage', 'transmission_automatic', 'transmission_manual',
       'transmission_other', 'drive_4wd', 'drive_fwd', 'drive_rwd',
       'size_compact', 'size_full-size', 'size_mid-size', 'size_sub-compact'],
      dtype='object')

### C отобранными признаками по методу RFE

In [18]:
name_col_RFE = ['cylinders_3', 'cylinders_4', 'cylinders_5', 'cylinders_6',
       'cylinders_8', 'cylinders_10','transmission_automatic', 'transmission_manual',
       'transmission_other','size_compact', 'size_full-size', 'size_mid-size', 'size_sub-compact']

In [19]:
model = LinearRegression()
model.fit(X_train[name_col_RFE], y_train)
y_predicted_RFE = model.predict(X_test[name_col_RFE])
 
mae = metrics.mean_absolute_error(y_test, y_predicted_RFE)
r2 = metrics.r2_score(y_test, y_predicted_RFE)
print('MAE: %.3f' % mae)
print('R2: %.3f' % r2)

MAE: 8060.058
R2: 0.132


### C отобранными признаками по методу SelectKBest

In [20]:
name_col_KB = ['year', 'condition', 'odometer']

In [21]:
model = LinearRegression()
model.fit(X_train[name_col_KB], y_train)
y_predicted_KB = model.predict(X_test[name_col_KB])
 
mae = metrics.mean_absolute_error(y_test, y_predicted_KB)
r2 = metrics.r2_score(y_test, y_predicted_KB)
print('MAE: %.3f' % mae)
print('R2: %.3f' % r2)

MAE: 5187.812
R2: 0.417


Итак, мы видим, что по метрикам (и МАЕ и R2) более лучший результат (причем значительно) показал отбор признаков по методу SelectKBest